In [ ]:
import torch
from torch import nn # contains PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"

print(torch.__version__)
print(f"Using device: {device}")

# Preparing Data

In [ ]:
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

In [ ]:
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split].to(device), y[:train_split].to(device)
X_test, y_test = X[train_split:].to(device), y[train_split:].to(device)

len(X_train), len(y_train), len(X_test), len(y_test)

# Predictions Graph

In [ ]:
def plot_predictions(train_data=X_train.cpu(), train_labels=y_train.cpu(), test_data=X_test.cpu(), test_labels=y_test.cpu(), predictions=None, title=None):
    plt.figure(figsize=(10, 7))
    plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")
    plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")
    
    if predictions is not None:
        plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")
        
    if title is not None:
        plt.title(title)
        
    plt.legend(prop={"size": 14})

In [ ]:
plot_predictions()

# Linear Regression Model
Used to build a standard linear regression model

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Initializes random weight and bias parameters
        self.linear_layer = nn.Linear(in_features=1, out_features=1)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

In [ ]:
torch.manual_seed(1)
model_0 = LinearRegressionModel().to(device)
list(model_0.parameters()), model_0.state_dict()

## Making Predictions

In [ ]:
with torch.inference_mode():
    y_preds = model_0(X_test)

### Check the Predictions

In [ ]:
print(f"Number of testing samples: {len(X_test)}")
print(f"Number of predictions made: {len(y_preds)}")
print(f"Predicted values:\n{y_preds}")

In [ ]:
plot_predictions(predictions=y_preds.cpu())

# Loss function and Optimizer

In [ ]:
loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.001) # lr is learning rate

# Testing and Training Loop

In [ ]:
torch.manual_seed(42)

# number of times the model will pass over the training data
epochs = 2000

train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    # Training
    
    # Model Training Mode
    model_0.train()
    
    # 1. Forward pass
    y_pred = model_0(X_train)
    
    # 2. Calculate loss
    loss = loss_fn(y_pred, y_train)
    
    # 3. Zero grad of the optimizer
    optimizer.zero_grad()
    
    # 4. Backpropagation on the loss
    loss.backward()
    
    # 5. Update optimizer
    optimizer.step()
    
    # Testing
    
    # Model Evaluation Mode
    model_0.eval()
    
    with torch.inference_mode():
        # 1. Forward pass
        test_pred = model_0(X_test)
        
        # 2. Calculate loss
        test_loss = loss_fn(test_pred, y_test.type(torch.float))
        
        if epoch % 100 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().cpu().numpy())
            test_loss_values.append(test_loss.detach().cpu().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss}")

# Plot Loss Curves

In [ ]:
plt.plot(epoch_count, train_loss_values, label="Train loss")
plt.plot(epoch_count, test_loss_values, label="Test loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend();

In [ ]:
print("The model learned the following values for weight and bias:")
print(f"Weight: {model_0.state_dict()['weight'][0]} | Bias: {model_0.state_dict()['bias'][0]}")
print("Original values:")
print(f"Weight: {weight} | Bias: {bias}")

In [ ]:
plot_predictions(predictions=y_preds.cpu(), title="Before Training")
with torch.inference_mode():
    y_preds = model_0(X_test)
plot_predictions(predictions=y_preds.cpu(), title="After Training")

# Saving a PyTorch model's state_dict()

In [ ]:
# 1. Create models directoy
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

#2. Create model save path
MODEL_NAME = "linear_regression_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

#3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(), f=MODEL_SAVE_PATH)

# Load model from state dict

In [ ]:
loaded_model_0 = LinearRegressionModel().to(device)

loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

In [ ]:
loaded_model_0.eval()

with torch.inference_mode():
    loaded_model_preds = loaded_model_0(X_test)

y_preds == loaded_model_preds